In [1]:
from googletrans import Translator
import numpy as np
import pandas as pd
import time
import gezi
from gezi import tqdm
import sys, os
from absl import flags
FLAGS = flags.FLAGS
sys.path.append('..')
sys.path.append('../../../../utils')
sys.path.append('../../../../third')
from src import util
pd.set_option('display.max_colwidth', 512)
gezi.init_flags()

No module named 'tensorflow.python.keras.layers.preprocessing'


In [2]:
root = '../input/feedback-prize-2021'

In [3]:
df = pd.read_feather(f'{root}/train_en.fea')

In [4]:
df

,index,id,part,para_id,para_type,start,end,start2,end2,para,text,pid,kfold,cluster,worker
0,0,423A1CA112E2,13,0,0,0,1,0,3.0,Phones [BR] [BR],"Phones [BR] [BR] Modern humans today are always on their phone. They are always on their phone more than 5 hours a day no stop .All they do is text back and forward and just have group Chats on social media. They even do it while driving. They are some really bad consequences when stuff happens when it comes to a phone. Some certain areas in the United States ban phones from class rooms just because of it. [BR] [BR] When people have phones, they know about certain apps that they have .Apps like Facebook...",0,2,8,67
1,1,423A1CA112E2,13,1,5,1,45,3,47.0,Modern humans today are always on their phone. They are always on their phone more than 5 hours a day no stop .All they do is text back and forward and just have group Chats on social media. They even do it while driving.,"Phones [BR] [BR] Modern humans today are always on their phone. They are always on their phone more than 5 hours a day no stop .All they do is text back and forward and just have group Chats on social media. They even do it while driving. They are some really bad consequences when stuff happens when it comes to a phone. Some certain areas in the United States ban phones from class rooms just because of it. [BR] [BR] When people have phones, they know about certain apps that they have .Apps like Facebook...",1,2,8,67
2,2,423A1CA112E2,13,2,3,45,60,47,62.0,They are some really bad consequences when stuff happens when it comes to a phone.,"Phones [BR] [BR] Modern humans today are always on their phone. They are always on their phone more than 5 hours a day no stop .All they do is text back and forward and just have group Chats on social media. They even do it while driving. They are some really bad consequences when stuff happens when it comes to a phone. Some certain areas in the United States ban phones from class rooms just because of it. [BR] [BR] When people have phones, they know about certain apps that they have .Apps like Facebook...",2,2,8,67
3,3,423A1CA112E2,13,3,2,60,76,62,80.0,Some certain areas in the United States ban phones from class rooms just because of it. [BR] [BR],"Phones [BR] [BR] Modern humans today are always on their phone. They are always on their phone more than 5 hours a day no stop .All they do is text back and forward and just have group Chats on social media. They even do it while driving. They are some really bad consequences when stuff happens when it comes to a phone. Some certain areas in the United States ban phones from class rooms just because of it. [BR] [BR] When people have phones, they know about certain apps that they have .Apps like Facebook...",3,2,8,67
4,4,423A1CA112E2,13,4,2,76,139,80,145.0,"When people have phones, they know about certain apps that they have .Apps like Facebook Twitter Instagram and Snapchat. So like if a friend moves away and you want to be in contact you can still be in contact by posting videos or text messages. People always have different ways how to communicate with a phone. Phones have changed due to our generation. [BR] [BR]","Phones [BR] [BR] Modern humans today are always on their phone. They are always on their phone more than 5 hours a day no stop .All they do is text back and forward and just have group Chats on social media. They even do it while driving. They are some really bad consequences when stuff happens when it comes to a phone. Some certain areas in the United States ban phones from class rooms just because of it. [BR] [BR] When people have phones, they know about certain apps that they have .Apps like Facebook...",4,2,8,67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175155,175155,4C471936CD75,7,10,2,576,828,582,834.0,"seeking multiple opinions before making a hard decision can be beneficial because of the reasons stated above. When I was in second grade, I passed the test to be admitted into the AAP (Advanced Academics

In [5]:
lang = 'zh-cn' if gezi.in_notebook() else sys.argv[1]
folds = 10
dlang = pd.concat(
  [pd.read_csv(f'{root}/trans_{lang}_{fold}.csv') for fold in range(folds) if os.path.exists(f'{root}/trans_{lang}_{fold}.csv')]
)
dlang['target'] = dlang['target'].apply(lambda x: x.replace('[Sep]', '[SEP]').replace('[SEP]', FLAGS.br))
dlang['mid'] = dlang['mid'].apply(lambda x: x.replace('[Sep]', '[SEP]').replace('[SEP]', FLAGS.br))
dlang = dlang.sort_values(['pid'])
dlang

,pid,text,mid,target
0,0,Phones [SEP] [SEP],手机[BR] [BR],Mobile phone [BR] [BR]
0,1,Modern humans today are always on their phone. They are always on their phone more than 5 hours a day no stop .All they do is text back and forward and just have group Chats on social media. They even do it while driving.,今天的现代人类总是在他们的手机上。他们总是在手机上超过5小时，没有停止。所有他们所做的是返回和前进的文字，只需在社交媒体上聊天。他们甚至在开车时这样做。,"Today's modern humans are always on their mobile phones.They always have more than 5 hours on the phone and have not stopped.All they do is back and advanced text, just chatting on social media.They did so even when driving."
0,2,They are some really bad consequences when stuff happens when it comes to a phone.,当涉及手机时，它们是一些非常糟糕的后果。,They are some very bad consequences when they involve mobile phones.
0,3,Some certain areas in the United States ban phones from class rooms just because of it. [SEP] [SEP],美国禁止来自课堂房间的某些地区，仅仅是因为它。[BR] [BR],The United States is prohibited from some areas from classroom rooms just because of it.[BR] [BR]
0,4,"When people have phones, they know about certain apps that they have .Apps like Facebook Twitter Instagram and Snapchat. So like if a friend moves away and you want to be in contact you can still be in contact by posting videos or text messages. People always have different ways how to communicate with a phone. Phones have changed due to our generation. [SEP] [SEP]",当人们有手机时，他们了解他们拥有的某些应用程序.Apps就像Facebook Twitter Instagram和Snapchat。所以，如果一个朋友离开，你想要联系你，你仍然可以通过发布视频或短信来联系。人们总是有不同的方式如何与手机沟通。由于我们的一代，手机已经改变。[BR] [BR],"When people have mobile phones, they understand some applications they have .Apps is like Facebook Twitter Instagram and SnapChat.So, if a friend leaves, you want to contact you, you can still contact your video or SMS.People always have different ways to communicate with mobile phones.Due to our generation, the phone has changed.[BR] [BR]"
...,...,...,...,...
16608,175153,"if I'm not sure what college I want to attend, and I ask one friend where I should go, they might say ""George Mason"" as a suggestion. But if George Mason is not in fact the best college for me, even though the person I asked was trying to help, that one opinion could sway me to go there anyway and not have the best experience. But yet if I go to ten people to advice, including the person who suggested George Mason, they might be the only one who would say that and maybe five of the other people I asked ...",如果我不确定我想参加哪所大学，我会问一位朋友应该去哪里，他们可能会说“乔治梅森”作为一个建议。但是，如果乔治梅森没有事实上是我最好的学院，即使我问的那个人试图提供帮助，那么一个意见可能会让我在那里去那里，而且没有最好的经历。但是，如果我到了十个人来建议，包括建议乔治梅森的人，他们可能是唯一一个人会说，也许我问的五个人可能会说弗吉尼亚科技，我会决定去弗吉尼亚技术而不是乔治梅森，我整体都会更开心，甚至可以获得更好的成绩。然后我会更成功，因为在我做出了可能影响我整个生命的决定之前，我去了多个人。因此，即使一个人仍然有机会提供有效的建议，与多个人交谈更安全，因为它增加了做出不良决定的机会。[BR] [BR],"If I don't sure I want to participate in which university, I will ask where a friend should go, they may say ""George Mason"" as a suggestion.However, if George Mason doesn't actually be my best college, even if I ask the person tried to help, then a comment may let me go there, and there is no best experience.However, if I reach ten people, it is recommended that the people who recommend George Mason may say that maybe some people may say Virginia Technology, I will decide to Virginia technology rather t..."
16627,175155,"seeking multiple opinions before making a hard decision can be beneficial because of the reasons stated above. When I was in second grade, I passed the test to be admitted into the AAP (Advanced Academics Program) in my school. My best friend was also accepted, and wanted me to go to AAP with her, but I didnt really want to go into AAP. I told my parents this, and they agreed it would be best for me to stay in the normal class for on more year. Since I got my parents' opinions, I also felt more secure s...",由于上述原因，在做出艰难的决定之前寻求多项意见可能是有益的。当我处于二年级时，我通过了考验，进入了学校的AAP（高级学者计划）。我最好的朋友也被接受了，并希望我和她一起去AAP，但我并没有真正想要进入AAP。我告诉我的父母，他们同意我最适合在普通的课程中留在普通阶段。既然我得到了父母的意见，我也感到更安全地留出了先进的计划，即使我的朋友希望我做相反并进入该计划。所以在三年级，我没有和我最好的朋友在一起，但尽管如此，我很高兴，在正常班上

In [6]:
dlang.to_csv(f'{root}/trans_{lang}.csv', index=False)

In [7]:
d = pd.merge(df, dlang[['pid', 'mid', 'target']], on='pid', how='left')
d = d.fillna('[NONE]')
d

,index,id,part,para_id,para_type,start,end,start2,end2,para,text,pid,kfold,cluster,worker,mid,target
0,0,423A1CA112E2,13,0,0,0,1,0,3.0,Phones [BR] [BR],"Phones [BR] [BR] Modern humans today are always on their phone. They are always on their phone more than 5 hours a day no stop .All they do is text back and forward and just have group Chats on social media. They even do it while driving. They are some really bad consequences when stuff happens when it comes to a phone. Some certain areas in the United States ban phones from class rooms just because of it. [BR] [BR] When people have phones, they know about certain apps that they have .Apps like Facebook...",0,2,8,67,手机[BR] [BR],Mobile phone [BR] [BR]
1,1,423A1CA112E2,13,1,5,1,45,3,47.0,Modern humans today are always on their phone. They are always on their phone more than 5 hours a day no stop .All they do is text back and forward and just have group Chats on social media. They even do it while driving.,"Phones [BR] [BR] Modern humans today are always on their phone. They are always on their phone more than 5 hours a day no stop .All they do is text back and forward and just have group Chats on social media. They even do it while driving. They are some really bad consequences when stuff happens when it comes to a phone. Some certain areas in the United States ban phones from class rooms just because of it. [BR] [BR] When people have phones, they know about certain apps that they have .Apps like Facebook...",1,2,8,67,今天的现代人类总是在他们的手机上。他们总是在手机上超过5小时，没有停止。所有他们所做的是返回和前进的文字，只需在社交媒体上聊天。他们甚至在开车时这样做。,"Today's modern humans are always on their mobile phones.They always have more than 5 hours on the phone and have not stopped.All they do is back and advanced text, just chatting on social media.They did so even when driving."
2,2,423A1CA112E2,13,2,3,45,60,47,62.0,They are some really bad consequences when stuff happens when it comes to a phone.,"Phones [BR] [BR] Modern humans today are always on their phone. They are always on their phone more than 5 hours a day no stop .All they do is text back and forward and just have group Chats on social media. They even do it while driving. They are some really bad consequences when stuff happens when it comes to a phone. Some certain areas in the United States ban phones from class rooms just because of it. [BR] [BR] When people have phones, they know about certain apps that they have .Apps like Facebook...",2,2,8,67,当涉及手机时，它们是一些非常糟糕的后果。,They are some very bad consequences when they involve mobile phones.
3,3,423A1CA112E2,13,3,2,60,76,62,80.0,Some certain areas in the United States ban phones from class rooms just because of it. [BR] [BR],"Phones [BR] [BR] Modern humans today are always on their phone. They are always on their phone more than 5 hours a day no stop .All they do is text back and forward and just have group Chats on social media. They even do it while driving. They are some really bad consequences when stuff happens when it comes to a phone. Some certain areas in the United States ban phones from class rooms just because of it. [BR] [BR] When people have phones, they know about certain apps that they have .Apps like Facebook...",3,2,8,67,美国禁止来自课堂房间的某些地区，仅仅是因为它。[BR] [BR],The United States is prohibited from some areas from classroom rooms just because of it.[BR] [BR]
4,4,423A1CA112E2,13,4,2,76,139,80,145.0,"When people have phones, they know about certain apps that they have .Apps like Facebook Twitter Instagram and Snapchat. So like if a friend moves away and you want to be in contact you can still be in contact by posting videos or text messages. People always have different ways how to communicate with a phone. Phones have changed due to our generation. [BR] [BR]","Phones [BR] [BR] Modern humans today are always on their phone. They are always on their phone more than 5 hours a day no stop .All they do is text back and forward and just have group Chats on social media. They even do it while driving. They are some really bad conseque

In [8]:
d.groupby('id').agg({
  'mid': list,
  'para_type': list,
}).reset_index().to_feather(f'{root}/{lang}.fea')

In [9]:
ic(len(d[d.mid != '[NONE]']) / len(d))

ic| 2511181817.py:1 in <module>
    len(d[d.mid != '[NONE]']) / len(d): 0.949537565654259


0.949537565654259

In [10]:
d[d.para =='[NONE]']

,index,id,part,para_id,para_type,start,end,start2,end2,para,text,pid,kfold,cluster,worker,mid,target


In [11]:
ids = []
texts = []
id = None
words = []
for row in tqdm(d.itertuples(), total=len(d)):
  row = row._asdict()
  if row['id'] != id:
    if words:
      ids.append(id)
      texts.append(' '.join(words))
    words = []
    id = row['id']
  col = 'para' if row['target'] == '[NONE]' else 'target'
  if len(str(row['para'].replace(FLAGS.br, '')).split(' ')) <= 2:
    col = 'para'
  words.extend(str(row[col]).split(' '))
if words:
  ids.append(id)
  texts.append(' '.join(words))
d_text = pd.DataFrame({
  'id': ids,
  'text': texts
})
d_text

  0%|          | 0/175160 [00:00<?, ?it/s]

,id,text
0,423A1CA112E2,"Mobile phone [BR] [BR] Today's modern humans are always on their mobile phones.They always have more than 5 hours on the phone and have not stopped.All they do is back and advanced text, just chatting on social media.They did so even when driving. They are some very bad consequences when they involve mobile phones. The United States is prohibited from some areas from classroom rooms just because of it.[BR] [BR] When people have mobile phones, they understand some applications they have .Apps is like Fac..."
1,A8445CABFECE,"Mobile phone and driving [BR] [BR] The driver should not use the phone when operating the vehicle. The driver who uses mobile phones when operating vehicles is most likely to enter may be a fatal accident.[BR] [BR] According to an article of Edgar Snyder, 21% of youth is part of a deadly car accident because of mobile phones.According to the same terms, 35% know the risk, but continue to use their phone on the road.This suggests that it exceeds dangerous and irresponsible drivers, rather than fully unde..."
2,6B4F7A0165B9,"Mobile phone operation when driving [BR] [BR] Although the ability to stay in the distance is initially implemented by using letters.It was found that the system was quite slow and searched for new ways, until the invention of telegrams; people believe this is a millennium invention, but it is considered to be slow in the invention of the phone after time.Today, the phone is in the hands of more than 700 billion people on the Earth or in your pocket. However, the device is brought to it is an irresponsi..."
3,E05C7F5C1156,"People are arguing that there should be no use or should not be used when operating the vehicle.Some people think that mobile phones should be allowed because they should try multitasking and have some calls, and some people addicted, and they cannot leave their mobile phones.Others say that because the phone may cause an accident during driving, you should not be on your phone. I also think that you should not use your mobile phone when you use your vehicle. Because it will cause the vehicle to collide..."
4,50B3435E475B,"Text and drive [BR] [BR] More than half of the driver in society has such a terrible habit of sending text messages and driving.This must stop immediately, send text messages and driving to make people's lives are risking.This simple behavior has led to death in the past few years.Fortunately, the government has put forward a new law and handles this problem by sending SMS and driving issues. I strongly think that texting and driving are dangerous and should be prohibited.[BR] [BR] Diving in more detail..."
...,...,...
15589,8F4B595CF9E7,"Do you need more comments and options when talking about recommendations?Ok, you can!You can simply ask more than an opinion for their opinions, so you can choose you! Go to multiple sources for advice, because It reaches new things, Get good or bad feedback, and You can now see what you can see from different perspectives.[BR] [BR] Seeking more comments can be open for new and different things. This can help you think about the box and form new thinking in some situations.Thinking outside the box can h..."
15590,6B5809C83978,"Does anyone give you suggestions?Is this contributing to this situation? Seeking a variety of comments can help others make better choices. Get different opinions Help someone make a better decision, because they will get multiple answers, Different types of recommendations, and They will have more information about the situation, then they start.[BR] [BR] By seeking a variety of comments, people will get different answers, which may be better.For example, I have encountered difficulties in deciding wha..."
15591,408A7D3D2EEC,"Imagine, seek multi-person suggestions and choose the best decisions that suits you.What do you feel?Requirements can be defined as people from multiplayers to help and make the right decisions. Seeking a variety of opinions can help others make be

In [12]:
d = pd.merge(d, d_text, on=['id'], how='inner', suffixes=('_ori', ''))

In [13]:
starts, ends = [], []
end = 0
for row in tqdm(d.itertuples(), total=len(d)):
  row = row._asdict()
  if row['para_id'] == 0:
    end = 0
  start = end
  col = 'para' if row['target'] == '[NONE]' else 'target'
  if len(str(row['para'].replace(FLAGS.br, '')).split(' ')) <= 2:
    col = 'para'
  end = start + len(str(row[col]).replace(FLAGS.br, '\n').split())
  starts.append(start)
  ends.append(end)
d['start'] = starts
d['end'] = ends

  0%|          | 0/175160 [00:00<?, ?it/s]

In [14]:
d

,index,id,part,para_id,para_type,start,end,start2,end2,para,text_ori,pid,kfold,cluster,worker,mid,target,text
0,0,423A1CA112E2,13,0,0,0,2,0,3.0,Phones [BR] [BR],"Phones [BR] [BR] Modern humans today are always on their phone. They are always on their phone more than 5 hours a day no stop .All they do is text back and forward and just have group Chats on social media. They even do it while driving. They are some really bad consequences when stuff happens when it comes to a phone. Some certain areas in the United States ban phones from class rooms just because of it. [BR] [BR] When people have phones, they know about certain apps that they have .Apps like Facebook...",0,2,8,67,手机[BR] [BR],Mobile phone [BR] [BR],"Mobile phone [BR] [BR] Today's modern humans are always on their mobile phones.They always have more than 5 hours on the phone and have not stopped.All they do is back and advanced text, just chatting on social media.They did so even when driving. They are some very bad consequences when they involve mobile phones. The United States is prohibited from some areas from classroom rooms just because of it.[BR] [BR] When people have mobile phones, they understand some applications they have .Apps is like Fac..."
1,1,423A1CA112E2,13,1,5,2,41,3,47.0,Modern humans today are always on their phone. They are always on their phone more than 5 hours a day no stop .All they do is text back and forward and just have group Chats on social media. They even do it while driving.,"Phones [BR] [BR] Modern humans today are always on their phone. They are always on their phone more than 5 hours a day no stop .All they do is text back and forward and just have group Chats on social media. They even do it while driving. They are some really bad consequences when stuff happens when it comes to a phone. Some certain areas in the United States ban phones from class rooms just because of it. [BR] [BR] When people have phones, they know about certain apps that they have .Apps like Facebook...",1,2,8,67,今天的现代人类总是在他们的手机上。他们总是在手机上超过5小时，没有停止。所有他们所做的是返回和前进的文字，只需在社交媒体上聊天。他们甚至在开车时这样做。,"Today's modern humans are always on their mobile phones.They always have more than 5 hours on the phone and have not stopped.All they do is back and advanced text, just chatting on social media.They did so even when driving.","Mobile phone [BR] [BR] Today's modern humans are always on their mobile phones.They always have more than 5 hours on the phone and have not stopped.All they do is back and advanced text, just chatting on social media.They did so even when driving. They are some very bad consequences when they involve mobile phones. The United States is prohibited from some areas from classroom rooms just because of it.[BR] [BR] When people have mobile phones, they understand some applications they have .Apps is like Fac..."
2,2,423A1CA112E2,13,2,3,41,52,47,62.0,They are some really bad consequences when stuff happens when it comes to a phone.,"Phones [BR] [BR] Modern humans today are always on their phone. They are always on their phone more than 5 hours a day no stop .All they do is text back and forward and just have group Chats on social media. They even do it while driving. They are some really bad consequences when stuff happens when it comes to a phone. Some certain areas in the United States ban phones from class rooms just because of it. [BR] [BR] When people have phones, they know about certain apps that they have .Apps like Facebook...",2,2,8,67,当涉及手机时，它们是一些非常糟糕的后果。,They are some very bad consequences when they involve mobile phones.,"Mobile phone [BR] [BR] Today's modern humans are always on their mobile phones.They always have more than 5 hours on the phone and have not stopped.All they do is back and advanced text, just chatting on social media.They did so even when driving. They are some very bad consequences when they involve mobile phones. The United States is prohibited from some areas from classroom rooms just because of it.[BR] [BR] When people have m

In [15]:
d[['id', 'para_type', 'start', 'end', 'text', 'kfold', 'worker']].reset_index().to_feather(f'{root}/train_{lang}.fea')

In [16]:
d.text.values[0]

"Mobile phone [BR] [BR] Today's modern humans are always on their mobile phones.They always have more than 5 hours on the phone and have not stopped.All they do is back and advanced text, just chatting on social media.They did so even when driving. They are some very bad consequences when they involve mobile phones. The United States is prohibited from some areas from classroom rooms just because of it.[BR] [BR] When people have mobile phones, they understand some applications they have .Apps is like Facebook Twitter Instagram and SnapChat.So, if a friend leaves, you want to contact you, you can still contact your video or SMS.People always have different ways to communicate with mobile phones.Due to our generation, the phone has changed.[BR] [BR] One way to drive how it arrives.People are always in their mobile phone during doing this.This can lead to serious problems. That's why there is no text message when driving.This is a very important thing.Some people still do this because the

In [17]:
gkeys = ['id', 'text', 'kfold', 'cluster', 'worker', 'part']
allkeys = gkeys + ['para_type', 'start', 'end']
dflat = d[allkeys].groupby(gkeys).agg(list).reset_index()
dflat

,id,text,kfold,cluster,worker,part,para_type,start,end
0,0000D23A521A,"Some people think that the so-called ""face"" on Mars is created by the lives of Mars.Not this.The face on Mars is a natural existence of Mesa. It is not created by aliens, and there is no conspiracy hidden on Mars Hide alien life.There is no evidence that NASA founded that this face was created by aliens.[BR] [BR] Mesa is a naturally occurring rock formation, found on Mars and Earth. This ""face"" on Mars looks like a face, because human beings tend to see the face, no matter where human beings are, human ...",0,7,10,2,"[3, 2, 2, 1, 6, 7, 2, 4]","[0, 26, 56, 68, 105, 121, 139, 164]","[26, 56, 68, 105, 121, 139, 164, 224]"
1,00066EA9880D,Unmanned cars is more intense to what they expect.The car driving in the case of no one is actually controlling the action of the vehicle on the wheel.The idea of unmanned cars into development has shown that the number of technologies manufactured by Wolrd is increased.The leader of this unmanned car is a car they call Google car.The arduous mission of creating a safe driver is not fully grasped. The development of these cars should be stopped immediately as it may have too many dangers and hazardous e...,4,5,9,1,"[5, 3, 0, 1, 2, 1, 2, 0, 1, 2, 4]","[0, 69, 88, 97, 112, 205, 219, 331, 346, 368, 447]","[69, 88, 97, 112, 205, 219, 331, 346, 368, 447, 504]"
2,000E6DE9E817,"Dear: Principal [BR] [BR] I argue policy change because Although some children really need to help their academic work, This doesn't mean that only their average, unless their average, it will not let them enjoy their sports or other activities.[BR] [BR] Sometimes teachers and even principals need to consider the help we should give any students to have. Today Self-regarded students as steam.It means that students will start feeling sad, not to go to school. Since they have a low average, they cannot pa...",3,13,3,0,"[0, 3, 0, 6, 7, 2, 0, 1, 2, 1, 0, 1, 0, 1, 0, 1, 0, 2, 4, 0]","[0, 2, 6, 7, 17, 38, 55, 56, 72, 102, 126, 131, 162, 173, 188, 189, 195, 196, 208, 213]","[2, 6, 7, 17, 38, 55, 56, 72, 102, 126, 131, 162, 173, 188, 189, 195, 196, 208, 213, 215]"
3,001552828BD0,"Can you give you a car?It has to ride strangers on the bus to reduce pollution.Can you pay a fine after driving a car?[BR] [BR] Most people will now give up their cars to save their lives.Since they come out, we have been accustomed to the car, we just forgot to use our bike, or from point A to Point B. ""70% of Vauban families do not have cars, while 57% of people sell this car""Say paragraph 3 of paragraph 1.Imagine selling your car, just in order to live in other places.No car will be one, reducing the...",3,11,78,15,"[5, 2, 1, 1, 2, 1, 2, 1, 2, 3]","[0, 24, 151, 163, 200, 256, 269, 323, 339, 425]","[24, 151, 163, 200, 256, 269, 323, 339, 425, 450]"
4,0016926B079C,"I think students will learn from home because They will not change and Get up in the morning, there is hair there. Only courses help them because there They will pay more attention. They will be comfortable at home.[BR] [BR] The most difficult part of the school is ready.You wake up to brush your teeth, then go to your closet, look at your cloth.After you think that you pick out your clothes, you will watch the mirror, you either you don't like it, or look and look at the stain.Then you have to change. ...",3,9,43,8,"[3, 1, 1, 0, 1, 1, 2, 1, 2, 1, 1, 2]","[0, 8, 13, 22, 28, 33, 39, 93, 109, 148, 179, 188]","[8, 13, 22, 28, 33, 39, 93, 109, 148, 179, 188, 226]"
...,...,...,...,...,...,...,...,...,...
15589,FFF1442D6698,"Every student looks forward to the summer vacation, which is about sunshine, out of fun.After a long time, they can stop at school for a while.Although, take a break, once they have to return to school, students often forget some knowledge.This is because of the summer, they just spend anything without schools.Therefore, a summer project was created.It helps students continue to learn and maintain their knowled

In [18]:
dflat.reset_index().to_feather(f'../input/feedback-prize-2021/train_flat_{lang}.fea')